# Mix Classes=Categories
version: 1

info: 
- mix/fusion of classes into other classes

author: nuno costa

In [1]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random

#Define root dir dependent on OS
rdir='D:/external_datasets/' 
if str(platform.platform()).find('linux')>-1: rdir='/mnt/d/external_datasets/' 
print('OS: {}'.format(platform.platform()))
print('root dir: {}'.format(rdir))

OS: Windows-10-10.0.20241-SP0
root dir: D:/external_datasets/


In [2]:
mlabjson =  json.load(open(rdir+'mlab_fix_equal.json'))

In [3]:
for k in mlabjson:
    print(k, len(mlabjson[k]))

info 5
licenses 9
categories 1261
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 1338002
datasets 2


## 1. Import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [47]:
# datasets name and id
dset_l=[]
dset_l_id=[]
for d in mlabjson['datasets']:
    dset_l.append(d['name'])
    dset_l_id.append(d['id'])
print(dset_l, dset_l_id)

['COCO', 'TAO'] [1, 2]


In [48]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in mlabjson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
#print(cat_l_id)

In [49]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in mlabjson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [50]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(mlabjson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    ann_dset.append(an['dataset'])

100%|█████████████████████████████████████████████████████████████████| 1338002/1338002 [00:03<00:00, 443120.17it/s]


## 2. Find mixers cat_ids
mixers example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [51]:
#mixers #TODO: SORT alphabetically
mixers_l=[]
mixers_l_catid=[]
mixers_l_catdset=[]
mixer_method="all_names"
if mixer_method=="all_names": #Do for all category names, even with equal 
    mixers_l=cat_l
    mixers_l_catid=[[id] for id in cat_l_id]
    mixers_l_catdset=[[dset] for dset in cat_l_dset]
    

print(mixers_l[0:5])
print(mixers_l_catid[0:5])
print(mixers_l_catdset[0:5])
print(len(mixers_l))
print(len(mixers_l_catid))
print(len(mixers_l_catdset))

['person', 'bicycle', 'car', 'motorcycle', 'airplane']
[[1], [2], [3], [4], [5]]
[['COCO'], ['COCO'], ['COCO'], ['COCO'], ['COCO']]
1261
1261
1261


In [52]:
# get annotations mixers
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
mixers_l_imgid=[]
mixers_l_bbox=[]
mixers_l_dset=[]
for catids in tqdm(mixers_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    mixers_l_imgid.append(l_imgid)
    mixers_l_bbox.append(l_bbox)
    mixers_l_dset.append(l_dset)

100%|███████████████████████████████████████████████████████████████████████████| 1261/1261 [00:13<00:00, 96.90it/s]


## 3. Classes|categories to fix w/ EXCEL report

In [53]:
classtomix_l=[]
classtomix_l_catid=[]
method="save_images"
if method=="save_images": # save images to folder for manual check with 
    datadir="mixers"
    folder=mixer_method+'/' #folder
    showimage=False #show images
    startidx=0 # start index of image to save from each dataset
    imgnr=5 # total number of images to save from each dataset
    imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
    path=os.path.join(rdir,datadir,folder)
    assure_path_exists(path) 
    df=pd.DataFrame({'mixers_l': mixers_l,'mixers_l_catid': mixers_l_catid, 'mixers_l_catdset': mixers_l_catdset, 'classtomix_l': np.nan, 'classtomix_l_catid':np.nan, 'rules':np.nan })
    df.loc[0, 'rules']="To fix classes: 1) You need to fill the column classtomix_l and/or classtomix_l_catid with the information from the respective mixer columns; 2) When copy/pasting or changing, make sure the same structure maintains:  ['car', 'carrot'], [3, 52], beware of the spaces ['car', '  and always maintain the first class in the list;  3) You have 3 possibilities of filling the columns : 1-the 2 columns empty, meaning the row will not be used for classtomix; 2-only one column empty, e.g. fill the classtotix_l row with the class labels from mixers_l, then during the importing the classtomix_l_catid is filled, and vice-versa; 3-If you want to change the name of the first class in the list,e.g ['car', 'carrot'] for ['automobile', 'carrot'] you need to provide the ids to classtomix_l_catid."
    excelpath=path+mixer_method+"_classtomix_report.xlsx"
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each mixer
    for i, mixer in enumerate(tqdm(mixers_l)): #run for each mixer category
        minclass=mixer
        if isinstance(minclass, list): minclass=minclass[0]
        print('\n>> '+minclass+'...') #class
        classpath=os.path.join(path, minclass) # folder for images of this class
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir, classpath, i, dset_l, mixers_l, mixers_l_catid, mixers_l_bbox, mixers_l_dset,
              mixers_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

  0%|                                                                                      | 0/1261 [00:00<?, ?it/s]


>> person...
COCO



  0%|                                                                              | 1/1261 [00:00<14:32,  1.44it/s]

[Errno 2] No such file or directory: 'D:/external_datasets/COCO/2017/images/val2017/000000037689.jpg'

>> bicycle...
COCO


  0%|                                                                              | 2/1261 [00:01<13:52,  1.51it/s]

...

[Errno 2] No such file or directory: 'D:/external_datasets/TAO/TAO_DIR/frames/val/YFCC100M/v_460a1314da9bd32cbd14a2b3414f30/frame1201.jpeg'




>>> Finding TAO wooden_spoon...:  83%|█████████████████████████████████████▌       | 30/36 [00:00<00:00, 188.98it/s]

[Errno 2] No such file or directory: 'D:/external_datasets/TAO/TAO_DIR/frames/val/YFCC100M/v_460a1314da9bd32cbd14a2b3414f30/frame1225.jpeg'



100%|███████████████████████████████████████████████████████████████████████████| 1261/1261 [05:32<00:00,  3.79it/s]



>> wreath...
>>>> #WARNING wreath is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wrench...
>>>> #WARNING wrench is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wristband...
>>>> #WARNING wristband is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wristlet...
>>>> #WARNING wristlet is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yacht...
>>>> #WARNING yacht is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yak...
>>>> #WARNING yak is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yogurt...
>>>> #WARNING yogurt is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yoke_(animal_e

In [45]:
# WARNING: CHECK EXCEL FIRST
if method=="save_images": # reading classtomix from excel
    classtomix_l=[]
    classtomix_l_catid=[]
    df=pd.read_excel(excelpath)
    classtomix_df=df.loc[:,'classtomix_l']
    classtomix_df_catid=df.loc[:,'classtomix_l_catid']
    display(df)
    # PARSE COLUMNS TO FIX
    classtomix_l=classtomix_df.tolist()
    classtomix_l_catid=classtomix_df_catid.tolist()
    #convert strings to lists
    for icl, cl in enumerate(classtomix_l): 
        if isinstance(classtomix_l[icl], str): classtomix_l[icl]=convert_unicode(classtomix_l[icl], method='liststr')
        if isinstance(classtomix_l_catid[icl], str): classtomix_l_catid[icl]=convert_unicode(classtomix_l_catid[icl], method='listnum')
    #parse the columns based on the rules    
    for ic, classes in enumerate(classtomix_df):
        #1. if only classtomix_l_catid empty - get 
        if not pd.isnull(classtomix_df.iloc[ic]) and pd.isnull(classtomix_df_catid.iloc[ic]):
            if len(classtomix_df.dropna()) == 2: # if only 2 class get remaining ids
                chosed_id=np.array([x for x in classtomix_l_catid if str(x) != 'nan' and str(x) !='[]'][0])
                remain_id=np.setdiff1d(np.array(cat_l_id), chosed_id)
                classtomix_l_catid[ic]=remain_id.tolist()
        #2. if only classtomix_l empty - raise
        if pd.isnull(classtomix_df.iloc[ic]) and not pd.isnull(classtomix_df_catid.iloc[ic]): 
            raise RuntimeError('only classtomix_l empty')

,mixers_l,mixers_l_catid,mixers_l_catdset,classtomix_l,classtomix_l_catid,rules,annotations_missing,images_missing
0,person,[1],['COCO'],aggressive,"[43, 44, 45]",To fix classes: 1) You need to fill the column...,[0],[0]
1,bicycle,[2],['COCO'],nonaggressive,NaN,NaN,[0],[0]
2,car,[3],['COCO'],NaN,NaN,NaN,[0],[0]
3,motorcycle,[4],['COCO'],NaN,NaN,NaN,[0],[0]
4,airplane,[5],['COCO'],NaN,NaN,NaN,[0],[0]
...,...,...,...,...,...,...,...,...
1256,yacht,[1305],['TAO'],NaN,NaN,NaN,[1],[0]
1257,yak,[1306],['TAO'],NaN,NaN,NaN,[1],[0]
1258,yogurt,[1307],['TAO'],NaN,NaN,NaN,[1],[0]
1259,yoke_(animal_equipment),[1308],['TAO'],NaN,NaN,NaN,[1],[0]


In [46]:
print('>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! \n')
fixempty=True
if fixempty:
    classtomix_l=[x for x in classtomix_l if str(x) != 'nan' and str(x) !='[]']
    classtomix_l_catid=[x for x in classtomix_l_catid if str(x) != 'nan' and str(x) !='[]']
print(classtomix_l)
print(classtomix_l_catid)

>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! 

[['aggressive'], ['nonaggressive']]
[[43, 44, 45], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 162, 163, 164, 165, 166, 167, 168, 169, 170, 172, 173, 174, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 205

## 4. Mix classes

In [26]:
# slow # newjson=copy.deepcopy(mlabjson) #do deepcopy to compare
# fast
newjson={'categories':[],'annotations':[] }
newjson['categories']=copy.copy(mlabjson['categories'])
newjson['annotations']=copy.copy(mlabjson['annotations'])

In [27]:
classtomix_l_catidx=[[cat_l_id.index(id) for id in id_l] for id_l in classtomix_l_catid]
#print(classtomix_l_catidx) # they should be less one, becacuse it is ordered
print((classtomix_l_catidx))

[[39, 42, 43, 44], [14, 15, 16]]


#### Change mlabjson['categories']: [{name: , id: }]  
=>  1. use first index cat id; 2. change name and change id;  remove the other categories (!!!Without ordering again the category id!!!)

In [28]:
# CHANGE NAME  & GET REMOVE List
keepidx_l=[]
keepid_l=[]
firstidx=0 # get first category id
for i,id_l in enumerate(tqdm(classtomix_l_catid)): #for each classtomix
    firstcatid=id_l[firstidx] # #category id 
    firstcatidx=classtomix_l_catidx[i][firstidx]# get cat index of first catid
    if isinstance(classtomix_l[i], list): newjson['categories'][firstcatidx]['name']=classtomix_l[i][firstidx] #change name of first id 
    else: newjson['categories'][firstcatidx]['name']=classtomix_l[i]
    assert newjson['categories'][firstcatidx]['id']==firstcatid #assert id - it should be the same
    keepidx_l.append(firstcatidx) #catidx to keep
    keepid_l.append(firstcatid) #catid to keep
keepidx_l=list(dict.fromkeys(keepidx_l)) # remove duplicates in the keep list
allidx_l=[index for index, value in enumerate(mlabjson['categories'])] # allidx in categories
removeidx_l=[idx for idx in allidx_l if idx not in keepidx_l] # remove idx 
removeitem_l=[newjson['categories'][removeidx] for removeidx in removeidx_l] #remove items #WARNING NECESSARY BECAUSE THE INDEX WILL CHANGE
print(len(allidx_l))
print(len(removeidx_l))
print(keepidx_l in removeidx_l)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

1310
1308
False


In [29]:
# REMOVE - newjson will be changed
for removeitem in removeitem_l: newjson['categories'].remove(removeitem)

In [30]:
print(newjson['categories'])

[{'supercategory': 'animal', 'id': 15, 'name': 'nonaggresive', 'dataset': 1}, {'supercategory': 'kitchen', 'id': 40, 'name': 'aggressive', 'dataset': 1}]


In [31]:
# GET NEW IDs - #WARNING after remove
ct_l_id=[]
for i,c in enumerate(tqdm(newjson['categories'])):
    ct_l_id.append(c['id'])
newidx_l=[ct_l_id.index(id) for id in keepid_l] # make sure same sequence of keepid_l #SAME ORDER AS EXCEL
newid_l=[i+1 for i in range(len(keepid_l))] #same sequence of keepid_l
print(keepid_l)
print(newidx_l)
print(newid_l)

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1960.41it/s]

[40, 15]
[1, 0]
[1, 2]


In [32]:
# SORT IDS - Excel order - newjson will be changed
categories_l=copy.copy(newjson['categories'])
for i,idx in enumerate(newidx_l):
    categories_l[idx]['id']=newid_l[i]
for i,idx in enumerate(newidx_l):
    newjson['categories'][i]=categories_l[idx] #TODO sort the id in the correct sequence

In [33]:
#TEST
print(categories_l)
print(newjson['categories'])
print(len(mlabjson['categories']))

[{'supercategory': 'animal', 'id': 2, 'name': 'nonaggresive', 'dataset': 1}, {'supercategory': 'kitchen', 'id': 1, 'name': 'aggressive', 'dataset': 1}]
[{'supercategory': 'kitchen', 'id': 1, 'name': 'aggressive', 'dataset': 1}, {'supercategory': 'animal', 'id': 2, 'name': 'nonaggresive', 'dataset': 1}]
1310


### QUESTION: REMOVE HYPERPARAMETERS? Mantain only id and NAME? OR irrelevant?

#### change mlabjson['annotations']: [{category_id: , }] 
=> 1.get annotation idx from catid; 2.update annotations id ; 3. update newjson['annotations']

In [34]:
# 1.get annotation idx from catid
ann_catid_np=np.array(ann_catid)
classtomix_l_ann_catidx=[[np.where(ann_catid_np==id)[0].tolist()  for id in id_l] for id_l in classtomix_l_catid]
print(classtomix_l)
print(classtomix_l_catid)
print(keepid_l)
print(newid_l)
print(len(classtomix_l_ann_catidx[0]))

[['aggressive'], ['nonaggresive']]
[[40, 43, 44, 45], [15, 16, 17]]
[40, 15]
[1, 2]
4


In [46]:
#2.update annotations id & 3. update newjson['annotations']
newjson['annotations']=copy.copy(mlabjson['annotations']) #reset annotations
copy_ann_l=copy.copy(newjson['annotations'])
newjson['annotations']=[] #clear
for i, ann_catidx_l in enumerate(classtomix_l_ann_catidx): 
    for catidx_l in ann_catidx_l:
        for catidx in catidx_l:
            copy_ann_l[catidx]['category_id']=newid_l[i] # update catid
            newjson['annotations'].append(copy_ann_l[catidx]) #update newjson (annotations sequence id will be lost
print(len(newjson['annotations']))

67517


In [48]:
#TEST
print(mlabjson['annotations'][8875])
print(newjson['annotations'][8875])

{'segmentation': [[20.95, 105.11, 20.95, 87.58, 8.43, 80.69, 8.43, 75.06, 38.48, 74.43, 40.35, 66.92, 32.22, 63.79, 39.1, 35.62, 135.51, 37.5, 130.5, 65.04, 128.0, 70.05, 124.87, 108.86, 119.86, 102.6, 119.23, 86.95, 111.72, 86.95, 106.08, 107.61, 104.83, 107.61, 98.57, 90.71, 77.91, 86.33, 64.77, 86.33, 55.38, 90.08, 45.99, 100.73, 45.36, 109.49, 41.61, 110.12, 40.35, 97.6, 35.97, 89.46, 29.71, 86.95, 24.7, 93.84, 24.7, 105.73]], 'area': 5697.957150000001, 'iscrowd': 0, 'image_id': 26929, 'bbox': [8.43, 35.62, 127.08, 74.5], 'category_id': 14, 'id': 8876, 'dataset': 1}
{'segmentation': [[163.18, 86.34, 165.19, 114.48, 160.5, 137.93, 151.12, 170.09, 155.14, 213.64, 175.91, 216.32, 191.32, 226.37, 195.34, 222.35, 198.02, 189.52, 198.02, 158.03, 194.0, 145.97, 188.64, 137.93, 185.29, 123.86, 183.28, 99.07, 183.28, 85.67, 181.94, 85.0, 163.85, 85.0]], 'area': 4522.6675, 'iscrowd': 0, 'image_id': 65785, 'bbox': [151.12, 85.0, 46.9, 141.37], 'category_id': 1, 'id': 281397, 'dataset': 1}


### 4. Save mixed json

In [50]:
# fast
mlabjson['categories']=copy.copy(newjson['categories'])
mlabjson['annotations']=copy.copy(newjson['annotations'])

In [51]:
# save
print('\n >> SAVING...')
jsonfile=rdir+'mlab_mix_aggressive.json'
with open(jsonfile, 'w') as f:
    json.dump(newjson, f)
print("JSON SAVED : {} \n".format(jsonfile))


 >> SAVING...
JSON SAVED : D:/external_datasets/mlabmix_aggressive.json 

